<a href="https://colab.research.google.com/github/getaccept/notebooks/blob/master/API_Export_document_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Export document data

Functions to export document meta data as CSV file


In [ ]:
# import dependencies
import requests
import re
from unicodedata import normalize
import os
import io
import csv
from google.colab import files
import ipywidgets as widgets

#Constants 
BASE_URL = "https://api.getaccept.com/v1"
SOURCE_ENTITY_ID = ""
#@title ↓↓ Click here to start
#@markdown This step might take a few seconds to run. <br>
#@markdown Then use __shift+enter__ key or click ► left of each step to go through the flow

In [ ]:
email_widget = widgets.Text(
    value="",
    placeholder="Enter login email",
    description="Email:",
    disabled=False
)
password_widget = widgets.Password(
    value="",
    placeholder="Enter password",
    description="Password:",
    disabled=False
)
#@markdown Use the form below to fill in login details to your entity in GetAccept and then run next cell to login
widgets.VBox([email_widget, password_widget])

In [ ]:
#@markdown Login and store API token
if email_widget.value and password_widget.value:
  payload = { "email": email_widget.value, "password": password_widget.value}
  if SOURCE_ENTITY_ID != "":
    payload["entity_id"] = SOURCE_ENTITY_ID
  response = requests.post(BASE_URL+"/auth", json=payload)
  data = response.json()
  if "access_token" in data:
    source_auth_headers = { "Authorization": "bearer " + data["access_token"]}
  else:
    raise TypeError(data["errors"], "Please check your credentials")
  # Check login and list entities
  response = requests.get(BASE_URL+"/users/me", headers=source_auth_headers)
  user_data = response.json()
  print("Logged in as " + user_data["user"]["first_name"] + " on entity " + user_data["user"]["entity_name"])
  SOURCE_ENTITY_ID = user_data["user"]["entity_id"]
else:
  raise TypeError("Could not login, missing email or password!")

In [ ]:
#@markdown Select the source entity you would like to get settings from. When you're done, run the next cell
source_entity_list = list(map(lambda x: (x["name"],x["id"]), user_data["entities"]))
source_entity_picker = widgets.Select(
    options=sorted(source_entity_list),
    value=SOURCE_ENTITY_ID,
)
source_entity_picker

In [ ]:
#@markdown Verifying entity token of source...
if source_entity_picker.value != SOURCE_ENTITY_ID:
  # Switch entity
  response = requests.get(BASE_URL+"/refresh/"+source_entity_picker.value, headers=source_auth_headers)
  data = response.json()
  if "access_token" in data:
    source_auth_headers = { "Authorization": "bearer " + data["access_token"]}
  SOURCE_ENTITY_ID = source_entity_picker.value
print("Authenticated to source entity \"%s\"" % source_entity_picker.label)

In [ ]:
document_count_widget = widgets.Text(
    value="500",
    placeholder="Enter document count",
    description="Count:",
    disabled=False
)
document_offset_widget = widgets.Text(
    value="0",
    placeholder="Enter document offset",
    description="Offset:",
    disabled=False
)
#@title Document Count & Offset
#@markdown Enter document count and offset. <br><hr>
#@markdown *Count* is the number of documents to get in the export<hr>
#@markdown *Offset* is the document number to start at. If you have collected data for the first 100 documents this should be 100 the next run.
widgets.VBox([document_count_widget, document_offset_widget])

In [ ]:
#@markdown Select the type of documents to get
document_type_picker = widgets.Select(
    options=[["Signed documents", "signed"], ["All documents", "all"]],
    value="signed",
)
document_type_picker

In [ ]:
#@markdown Enter export filename
#@markdown *ex. export.csv*<br>
def slugify(text, delim='-'):
    """Generates an slightly worse ASCII-only slug."""
    result = []
    punct_re = re.compile(r'[\t !"#$%&\'()*\-/<=>?@\[\\\]^_`{|},.:]+')
    for word in punct_re.split(text.lower()):
        word = normalize('NFKD', word).encode('ascii', 'ignore')
        word = word.decode('utf-8')
        if word:
            result.append(word)
    return delim.join(result)
suggested_filename = slugify(source_entity_picker.label, "_") + "-" + slugify(document_type_picker.label, "_") + ".csv"
export_filename_widget = widgets.Text(
    value=suggested_filename,
    placeholder="Enter filename",
    description="Filename:",
    layout={"width": "500px"},
    disabled=False
)
export_filename_widget

In [ ]:
#@title Get document list
#@markdown Preparing documents for export
DOCUMENT_COUNT = int(document_count_widget.value)
DOCUMENT_OFFSET = int(document_offset_widget.value)
EXPORT_FILENAME = export_filename_widget.value
if document_type_picker.value == "all":
  # Get data for all documents
  response = requests.get(BASE_URL+"/documents?showall=true&sort_by=created&sort_order=desc&offset="+str(DOCUMENT_OFFSET)+"&limit="+str(DOCUMENT_COUNT),headers=source_auth_headers)
  document_data = response.json()
if document_type_picker.value == "signed":
  # Get data for signed documents
  response = requests.get(BASE_URL+"/documents?filter=signed&showall=true&sort_order=desc&offset="+str(DOCUMENT_OFFSET)+"&limit="+str(DOCUMENT_COUNT),headers=source_auth_headers)
  document_data = response.json()

# Get field names and flatten recipients
fieldnames = []
for document in document_data:
  for key, field in enumerate(document):
    if field not in fieldnames and not isinstance(document[field],list):
      fieldnames.append(field)

  if 'recipients' in document:
    for recid,recipient in enumerate(document['recipients']):
      for key, value in recipient.items():
        fieldname = 'recipient{}_{}'.format(recid+1,key)
        document[fieldname] = value
        if not fieldname in fieldnames:
          fieldnames.append(fieldname)
  
    del document['recipients']
print("%d documents ready to export" % len(document_data))

In [ ]:
#@title Export and download CSV file with data
#@markdown The file will automatically be downloaded after export is finished.
with open(EXPORT_FILENAME, 'w', newline='') as csv_file:
    # Attach a CSV writer to the file with the desired fieldnames
    writer = csv.DictWriter(csv_file, fieldnames)
    # Write the header row
    writer.writeheader()

    # Write document data to rows
    for row in document_data:
      writer.writerow(row)
files.download(EXPORT_FILENAME) 
